In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
pet = pd.read_csv("train.csv")
pet.head()

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2,breed_category,pet_category
0,ANSL_69903,2016-07-10 00:00:00,2016-09-21 16:25:00,2.0,Brown Tabby,0.80,7.78,13,9,0.0,1
1,ANSL_66892,2013-11-21 00:00:00,2018-12-27 17:47:00,1.0,White,0.72,14.19,13,9,0.0,2
2,ANSL_69750,2014-09-28 00:00:00,2016-10-19 08:24:00,NaN,Brown,0.15,40.90,15,4,2.0,4
3,ANSL_71623,2016-12-31 00:00:00,2019-01-25 18:30:00,1.0,White,0.62,17.82,0,1,0.0,2
4,ANSL_57969,2017-09-28 00:00:00,2017-11-19 09:38:00,2.0,Black,0.50,11.06,18,4,0.0,1


In [3]:
pet.drop(columns=["pet_id","issue_date","listing_date"],inplace=True)

In [4]:
pet["color"]= pd.factorize(pet["color_type"])[0]
pet.drop(columns=["color_type"], inplace=True)

In [5]:
pet["height(m)"] = pet["height(cm)"]/100
pet.drop(columns=["height(cm)"],inplace=True)

In [6]:
pet.head()

,condition,length(m),X1,X2,breed_category,pet_category,color,height(m)
0,2.0,0.80,13,9,0.0,1,0,0.0778
1,1.0,0.72,13,9,0.0,2,1,0.1419
2,NaN,0.15,15,4,2.0,4,2,0.4090
3,1.0,0.62,0,1,0.0,2,1,0.1782
4,2.0,0.50,18,4,0.0,1,3,0.1106


In [7]:
pet.isnull().sum()

condition         1477
length(m)            0
X1                   0
X2                   0
breed_category       0
pet_category         0
color                0
height(m)            0
dtype: int64

In [8]:
to_fill = pet[pet["condition"].isnull()].copy()
newPet = pet[~pet["condition"].isnull()].copy()

In [9]:
to_fill.reset_index(inplace=True)

In [10]:
to_fill.isnull().sum()

index                0
condition         1477
length(m)            0
X1                   0
X2                   0
breed_category       0
pet_category         0
color                0
height(m)            0
dtype: int64

In [11]:
to_fill.head()

,index,condition,length(m),X1,X2,breed_category,pet_category,color,height(m)
0,2,NaN,0.15,15,4,2.0,4,2,0.4090
1,21,NaN,0.45,11,4,2.0,4,2,0.1400
2,35,NaN,0.03,13,9,2.0,1,1,0.2190
3,38,NaN,0.73,0,6,2.0,4,2,0.0822
4,43,NaN,0.08,9,2,2.0,1,3,0.3040


In [12]:
to_fill.drop(columns=["index","condition"], inplace=True)

In [13]:
newPet.reset_index(inplace=True)
newPet.drop(columns=["index"], inplace=True)

In [14]:
newPet.head()

,condition,length(m),X1,X2,breed_category,pet_category,color,height(m)
0,2.0,0.80,13,9,0.0,1,0,0.0778
1,1.0,0.72,13,9,0.0,2,1,0.1419
2,1.0,0.62,0,1,0.0,2,1,0.1782
3,2.0,0.50,18,4,0.0,1,3,0.1106
4,0.0,0.92,0,7,1.0,2,4,0.2956


In [15]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split

In [16]:
x = newPet.drop(columns=["condition"])
y = newPet["condition"]

In [17]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [18]:
etc = GradientBoostingClassifier(random_state=42)
etc.fit(x_train,y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=42, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [19]:
etc.score(x_test,y_test)

0.8168202764976958

In [20]:
ans = pd.DataFrame(etc.predict(to_fill))

In [21]:
ans.head()

,0
0,2.0
1,2.0
2,2.0
3,2.0
4,2.0


In [22]:
ans[0].unique()

array([2., 0.])

In [23]:
to_fill.head()

,length(m),X1,X2,breed_category,pet_category,color,height(m)
0,0.15,15,4,2.0,4,2,0.4090
1,0.45,11,4,2.0,4,2,0.1400
2,0.03,13,9,2.0,1,1,0.2190
3,0.73,0,6,2.0,4,2,0.0822
4,0.08,9,2,2.0,1,3,0.3040


In [24]:
ans.rename(columns={0:"condition"},inplace=True)
ans = pd.concat([ans,to_fill],axis=1)

In [25]:
to_fill=ans

In [26]:
to_fill.head()

,condition,length(m),X1,X2,breed_category,pet_category,color,height(m)
0,2.0,0.15,15,4,2.0,4,2,0.4090
1,2.0,0.45,11,4,2.0,4,2,0.1400
2,2.0,0.03,13,9,2.0,1,1,0.2190
3,2.0,0.73,0,6,2.0,4,2,0.0822
4,2.0,0.08,9,2,2.0,1,3,0.3040


In [27]:
to_fill.isnull().sum()

condition         0
length(m)         0
X1                0
X2                0
breed_category    0
pet_category      0
color             0
height(m)         0
dtype: int64

In [28]:
newPet.head()

,condition,length(m),X1,X2,breed_category,pet_category,color,height(m)
0,2.0,0.80,13,9,0.0,1,0,0.0778
1,1.0,0.72,13,9,0.0,2,1,0.1419
2,1.0,0.62,0,1,0.0,2,1,0.1782
3,2.0,0.50,18,4,0.0,1,3,0.1106
4,0.0,0.92,0,7,1.0,2,4,0.2956


In [29]:
to_fill.head()

,condition,length(m),X1,X2,breed_category,pet_category,color,height(m)
0,2.0,0.15,15,4,2.0,4,2,0.4090
1,2.0,0.45,11,4,2.0,4,2,0.1400
2,2.0,0.03,13,9,2.0,1,1,0.2190
3,2.0,0.73,0,6,2.0,4,2,0.0822
4,2.0,0.08,9,2,2.0,1,3,0.3040


In [30]:
newPet.shape,to_fill.shape

((17357, 8), (1477, 8))

In [31]:
newPet = pd.concat([newPet, to_fill])

In [32]:
newPet.head()

,condition,length(m),X1,X2,breed_category,pet_category,color,height(m)
0,2.0,0.80,13,9,0.0,1,0,0.0778
1,1.0,0.72,13,9,0.0,2,1,0.1419
2,1.0,0.62,0,1,0.0,2,1,0.1782
3,2.0,0.50,18,4,0.0,1,3,0.1106
4,0.0,0.92,0,7,1.0,2,4,0.2956


In [33]:
newPet.shape

(18834, 8)

In [34]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split

In [35]:
x = newPet.drop(columns=["breed_category", "pet_category"])
y = newPet["breed_category"]

In [36]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [37]:
gdc = GradientBoostingClassifier(random_state=42)

In [38]:
gdc.fit(x_train,y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=42, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [39]:
gdc.score(x_test,y_test)

0.8606318024953544

In [40]:
c = pd.DataFrame(gdc.predict(x_test))
c.head()

,0
0,1.0
1,0.0
2,0.0
3,0.0
4,1.0


In [41]:
t = pd.DataFrame(y_test).reset_index()
ans =t
ans["pre_breed"] = c

In [42]:
ans.drop(columns=["index"],inplace=True)

In [43]:
ans.head(10)

,breed_category,pre_breed
0,1.0,1.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,1.0,1.0
5,2.0,2.0
6,0.0,0.0
7,1.0,1.0
8,1.0,0.0
9,0.0,0.0


In [44]:
GDC = GradientBoostingClassifier(random_state=42)

In [45]:
X = newPet.drop(columns=["breed_category", "pet_category"])
Y = newPet["pet_category"]
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=42)
GDC.fit(X_train,Y_train)
c = pd.DataFrame(GDC.predict(X_test))

In [46]:
ans.head()

,breed_category,pre_breed
0,1.0,1.0
1,0.0,0.0
2,0.0,0.0
3,0.0,0.0
4,1.0,1.0


In [47]:
ans2 = pd.DataFrame(Y_test).reset_index()
ans2["pred_pet"] = c

In [48]:
ans2.head()

,index,pet_category,pred_pet
0,10067,2,2
1,5921,2,2
2,6847,2,2
3,1383,1,1
4,3682,1,1


In [49]:
ans2.drop(columns=["index"], inplace=True)

In [50]:
ans2.head()

,pet_category,pred_pet
0,2,2
1,2,2
2,2,2
3,1,1
4,1,1


In [51]:
ans = pd.concat([ans,ans2],axis=1)

In [52]:
ans.head()

,breed_category,pre_breed,pet_category,pred_pet
0,1.0,1.0,2,2
1,0.0,0.0,2,2
2,0.0,0.0,2,2
3,0.0,0.0,1,1
4,1.0,1.0,1,1


In [53]:
gdc.score(X_test,Y_test)

0.20069020440668967

## END OF TRAINING 

In [54]:
test = pd.read_csv("test.csv")
test.head()

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2
0,ANSL_75005,2005-08-17 00:00:00,2017-09-07 15:35:00,0.0,Black,0.87,42.73,0,7
1,ANSL_76663,2018-11-15 00:00:00,2019-05-08 17:24:00,1.0,Orange Tabby,0.06,6.71,0,1
2,ANSL_58259,2012-10-11 00:00:00,2018-04-02 16:51:00,1.0,Black,0.24,41.21,0,7
3,ANSL_67171,2015-02-13 00:00:00,2018-04-06 07:25:00,1.0,Black,0.29,8.46,7,1
4,ANSL_72871,2017-01-18 00:00:00,2018-04-26 13:42:00,1.0,Brown,0.71,30.92,0,7


In [55]:
test["height(m)"] = test["height(cm)"]/100
test.drop(columns=["height(cm)"],inplace=True)

In [56]:
ntest = test.copy()

In [57]:
ntest.drop(columns=["pet_id", "issue_date", "listing_date"],inplace=True)

In [58]:
ntest.head()

,condition,color_type,length(m),X1,X2,height(m)
0,0.0,Black,0.87,0,7,0.4273
1,1.0,Orange Tabby,0.06,0,1,0.0671
2,1.0,Black,0.24,0,7,0.4121
3,1.0,Black,0.29,7,1,0.0846
4,1.0,Brown,0.71,0,7,0.3092


In [59]:
ntest.head()

,condition,color_type,length(m),X1,X2,height(m)
0,0.0,Black,0.87,0,7,0.4273
1,1.0,Orange Tabby,0.06,0,1,0.0671
2,1.0,Black,0.24,0,7,0.4121
3,1.0,Black,0.29,7,1,0.0846
4,1.0,Brown,0.71,0,7,0.3092


In [60]:
len(ntest["color_type"].unique())

54

In [61]:
ntest["color"] = pd.factorize(ntest["color_type"])[0]
ntest.drop(columns=["color_type"],inplace=True)

In [62]:
ntest.head()

,condition,length(m),X1,X2,height(m),color
0,0.0,0.87,0,7,0.4273,0
1,1.0,0.06,0,1,0.0671,1
2,1.0,0.24,0,7,0.4121,0
3,1.0,0.29,7,1,0.0846,0
4,1.0,0.71,0,7,0.3092,2


In [63]:
na = ntest[ntest["condition"].isnull()].copy()
sim = ntest[~ntest["condition"].isnull()].copy()

In [64]:
na.head()

,condition,length(m),X1,X2,height(m),color
19,NaN,0.94,15,4,0.3424,2
27,NaN,0.34,15,4,0.2362,12
31,NaN,0.96,15,4,0.0568,2
35,NaN,0.41,18,4,0.1861,12
40,NaN,0.74,16,9,0.3520,0


In [65]:
na.reset_index(inplace=True)

In [66]:
na.drop(columns=["condition","index"],inplace=True)

In [67]:
sim.reset_index(inplace=True)
sim.drop(columns=["index"],inplace=True)

In [68]:
xt = sim.drop(columns=["condition"])
yt = sim["condition"]
xtrain,xtest,ytrain,ytest = train_test_split(xt, yt, test_size=0.2, random_state=42)

In [69]:
etc.fit(xtrain, ytrain)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=42, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [70]:
etc.score(xtest,ytest)

0.5070422535211268

In [71]:
testans = pd.DataFrame(etc.predict(na))

In [72]:
testans.head()

,0
0,2.0
1,2.0
2,2.0
3,2.0
4,2.0


In [73]:
testans = pd.concat([testans, na],axis=1)

In [74]:
testans.head()

,0,length(m),X1,X2,height(m),color
0,2.0,0.94,15,4,0.3424,2
1,2.0,0.34,15,4,0.2362,12
2,2.0,0.96,15,4,0.0568,2
3,2.0,0.41,18,4,0.1861,12
4,2.0,0.74,16,9,0.3520,0


In [75]:
testans.rename(columns={0:"condition"}, inplace=True)

In [76]:
testans.isnull().sum()

condition    0
length(m)    0
X1           0
X2           0
height(m)    0
color        0
dtype: int64

In [77]:
testans.head()

,condition,length(m),X1,X2,height(m),color
0,2.0,0.94,15,4,0.3424,2
1,2.0,0.34,15,4,0.2362,12
2,2.0,0.96,15,4,0.0568,2
3,2.0,0.41,18,4,0.1861,12
4,2.0,0.74,16,9,0.3520,0


In [78]:
sim = pd.concat([sim,testans])

In [79]:
sim.head()

,condition,length(m),X1,X2,height(m),color
0,0.0,0.87,0,7,0.4273,0
1,1.0,0.06,0,1,0.0671,1
2,1.0,0.24,0,7,0.4121,0
3,1.0,0.29,7,1,0.0846,0
4,1.0,0.71,0,7,0.3092,2


In [80]:
sim.isnull().sum()

condition    0
length(m)    0
X1           0
X2           0
height(m)    0
color        0
dtype: int64

In [81]:
ans1 = pd.DataFrame(gdc.predict(sim))
ans1.rename(columns={0:"breed_category"},inplace=True)
ans1["breed_category"].astype(int)

0       1
1       0
2       0
3       0
4       0
       ..
8067    0
8068    0
8069    2
8070    0
8071    0
Name: breed_category, Length: 8072, dtype: int64

In [82]:
ans2 = pd.DataFrame(GDC.predict(sim))
ans2.rename(columns={0:"pet_category"},inplace=True)
ans2["pet_category"].astype(int)

0       1
1       1
2       1
3       1
4       1
       ..
8067    1
8068    1
8069    1
8070    1
8071    1
Name: pet_category, Length: 8072, dtype: int64

In [83]:
test.head()

,pet_id,issue_date,listing_date,condition,color_type,length(m),X1,X2,height(m)
0,ANSL_75005,2005-08-17 00:00:00,2017-09-07 15:35:00,0.0,Black,0.87,0,7,0.4273
1,ANSL_76663,2018-11-15 00:00:00,2019-05-08 17:24:00,1.0,Orange Tabby,0.06,0,1,0.0671
2,ANSL_58259,2012-10-11 00:00:00,2018-04-02 16:51:00,1.0,Black,0.24,0,7,0.4121
3,ANSL_67171,2015-02-13 00:00:00,2018-04-06 07:25:00,1.0,Black,0.29,7,1,0.0846
4,ANSL_72871,2017-01-18 00:00:00,2018-04-26 13:42:00,1.0,Brown,0.71,0,7,0.3092


In [84]:
test = pd.concat([test,ans1,ans2],axis=1)

In [85]:
test.head()

,pet_id,issue_date,listing_date,condition,color_type,length(m),X1,X2,height(m),breed_category,pet_category
0,ANSL_75005,2005-08-17 00:00:00,2017-09-07 15:35:00,0.0,Black,0.87,0,7,0.4273,1.0,1
1,ANSL_76663,2018-11-15 00:00:00,2019-05-08 17:24:00,1.0,Orange Tabby,0.06,0,1,0.0671,0.0,1
2,ANSL_58259,2012-10-11 00:00:00,2018-04-02 16:51:00,1.0,Black,0.24,0,7,0.4121,0.0,1
3,ANSL_67171,2015-02-13 00:00:00,2018-04-06 07:25:00,1.0,Black,0.29,7,1,0.0846,0.0,1
4,ANSL_72871,2017-01-18 00:00:00,2018-04-26 13:42:00,1.0,Brown,0.71,0,7,0.3092,0.0,1


In [86]:
test = test.drop(columns=["issue_date","listing_date","condition","color_type","length(m)","height(m)","X1","X2"])

In [87]:
test.head()

,pet_id,breed_category,pet_category
0,ANSL_75005,1.0,1
1,ANSL_76663,0.0,1
2,ANSL_58259,0.0,1
3,ANSL_67171,0.0,1
4,ANSL_72871,0.0,1


In [88]:
test.head()

,pet_id,breed_category,pet_category
0,ANSL_75005,1.0,1
1,ANSL_76663,0.0,1
2,ANSL_58259,0.0,1
3,ANSL_67171,0.0,1
4,ANSL_72871,0.0,1


In [90]:
test.to_csv("Pet.csv",index=False)